In [2]:
import pandas as pd
import numpy as np
from random import randint
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

#load data from memory
data=pd.read_pickle('/scratch/lnw8px/GAN/mydata.pkl')

In [3]:
data.head()

,gesture,x,y,z
0,1,"[-6.981482, -6.86912452173913, -6.934154217391...","[1.7387842, 1.7656500521739131, 1.719415726086...","[7.6556797, 7.653805534782609, 7.6723408695652..."
1,1,"[-7.3263645, -7.1698902, -7.164701, -7.2293663...","[1.1735595, 1.2569860666666668, 1.094124766666...","[7.723938, 7.5858251, 7.535130666666666, 7.547..."
2,1,"[-7.204219, -6.959433458823529, -7.10024679411...","[1.4034815, 1.3367731705882353, 1.344733, 1.23...","[7.662865, 7.678221223529412, 7.61806400588235..."
3,1,"[-7.2832546, -7.261205917647059, -7.2083745882...","[1.2142749, 1.1385500629411764, 1.209132623529...","[7.597002, 7.3560911764705885, 7.3024145882352..."
4,1,"[-8.074808, -7.943681, -7.7766285, -7.65987134...","[1.6896863, 1.6208295, 1.8238074, 2.57584345, ...","[6.724017, 6.69288215, 6.6306114, 6.7090485, 7..."


In [4]:
def get_max(array):
    return np.max(array)
def get_min(array):
    return np.min(array)
print(np.max(data['x'].apply(get_max).values),np.min(data['x'].apply(get_min).values))
print(np.max(data['y'].apply(get_max).values),np.min(data['y'].apply(get_min).values))
print(np.max(data['z'].apply(get_max).values),np.min(data['z'].apply(get_min).values))

-0.9769284200000035 -11.142999714285715
8.90588 -10.572148333333335
18.893112 -5.80462875


In [5]:
'''
normalize data
based on max and min values found on the dataprep section
'''

'\nnormalize data\nbased on max and min values found on the dataprep section\n'

In [6]:
def get_norm_data(array,val):
    return list(np.array(array)/val)

data['x_norm']=data['x'].apply(get_norm_data,args=[12])
data['y_norm']=data['y'].apply(get_norm_data,args=[11])
data['z_norm']=data['z'].apply(get_norm_data,args=[19])

In [7]:
'''
create onehot labels
'''

'\ncreate onehot labels\n'

In [8]:
num_classes=20
def to_onehot(num):
    onehot = np.zeros(num_classes,dtype=int)
    onehot[num-1] = 1
    return onehot

data['label']=data['gesture'].apply(to_onehot)

In [9]:
'''
crop data
'''

'\ncrop data\n'

In [10]:
def crop_array(array):
    #crop_len/10 is the length in seconds
    crop_len=30
    #pad array if its shorter than crop_len
    if(len(array)<crop_len):
        a=np.pad(array,pad_width=[0,(crop_len-len(array))],mode='edge')
    else:
        max_start=len(array)-crop_len
        start=randint(0,max_start)
        a=array[start:start+crop_len]
    return a 

crop_x=data['x_norm'].apply(crop_array)
crop_y=data['y_norm'].apply(crop_array)
crop_z=data['z_norm'].apply(crop_array)

data['crop_x']=crop_x
data['crop_y']=crop_y
data['crop_z']=crop_z

In [11]:
'''
select only the required columns
'''

'\nselect only the required columns\n'

In [12]:
data=data.drop(columns=['x','y','z','x_norm','y_norm','z_norm'])
data.head()

,gesture,label,crop_x,crop_y,crop_z
0,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.5817901666666666, -0.5724270434782609, -0....","[0.15807129090909092, 0.16051364110671937, 0.1...","[0.4029305105263158, 0.40283187025171624, 0.40..."
1,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.6105303750000001, -0.59749085, -0.59705841...","[0.10668722727272728, 0.11427146060606062, 0.0...","[0.40652305263157895, 0.399253952631579, 0.396..."
2,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.6003515833333334, -0.5799527882352941, -0....","[0.12758922727272728, 0.12152483368983957, 0.1...","[0.4033086842105263, 0.40411690650154797, 0.40..."
3,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.6069378833333333, -0.6051004931372549, -0....","[0.11038862727272726, 0.10350455117647059, 0.1...","[0.3998422105263158, 0.387162693498452, 0.3843..."
4,1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.6729006666666667, -0.6619734166666666, -0....","[0.15360784545454545, 0.14734813636363636, 0.1...","[0.3538956315789474, 0.35225695526315787, 0.34..."


In [13]:
def get_max(array):
    return np.max(array)
def get_min(array):
    return np.min(array)
print(np.max(data['crop_x'].apply(get_max).values),np.min(data['crop_x'].apply(get_min).values))
print(np.max(data['crop_y'].apply(get_max).values),np.min(data['crop_y'].apply(get_min).values))
print(np.max(data['crop_z'].apply(get_max).values),np.min(data['crop_z'].apply(get_min).values))

-0.08141070166666696 -0.9285833095238095
0.8096254545454545 -0.9611043939393941
0.9943743157894737 -0.3055067763157895


In [14]:
'''
seperate train and test data.
one user is test. All others is train
'''

'\nseperate train and test data.\none user is test. All others is train\n'

In [15]:
def scale(row):
    scaled=(row-np.min(row))/(np.max(row)-np.min(row))
    return scaled

def get_batch(data,batch_size):
    feature_list=[]
    label_list=[]
    batch=data.sample(n=batch_size)
    for i in range(len(batch)):
        features=[]
        features.append(batch.iloc[i]['crop_x'])
        features.append(batch.iloc[i]['crop_y'])
        features.append(batch.iloc[i]['crop_z'])
        features=np.array(features)
        feature_list.append(features)
        label_list.append(batch.iloc[i]['gesture']-1)
    features=np.array(feature_list)
    features=torch.from_numpy(features)
    features=features.to(device)
    features=features.float()
    labels=np.array(label_list)
    labels=torch.from_numpy(labels)
    labels=labels.to(device)
    labels=labels.long()
    return features,labels

def get_generated_batch(batch_size):
    noise = torch.randn(batch_size,100).to(device)
    labels = torch.randint(0, 20, (batch_size,)).to(device)
    generated=generator(noise,labels)
    
    return generated,labels

def get_comb_batch(data,real_bs,gen_bs):
    real_features,real_labels=get_batch(data=data,batch_size=real_bs)
    gen_features,gen_labels=get_generated_batch(gen_bs)
    features=torch.cat([real_features,gen_features],0)
    ar=features.detach().cpu().numpy()
    labels=torch.cat([real_labels,gen_labels],0)
    return features,labels

In [ ]:
features,labels=get_comb_batch(data,10,12)

In [18]:
features.shape

torch.Size([22, 3, 30])

In [13]:
'''
Define the CNN
'''

'\nDefine the CNN\n'

In [18]:
def get_model(num_filters1,num_filters2): 
    
    features,labels=get_batch(data,batch_size=10)
    conv1 = (nn.Conv1d(in_channels=3, out_channels=num_filters1, kernel_size=2, stride=1).float()).to(device)
    pool1 = (nn.MaxPool1d(kernel_size=2).float()).to(device)
    
    conv2 = (nn.Conv1d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=2, stride=1).float()).to(device)
    pool2 = (nn.MaxPool1d(kernel_size=2).float()).to(device)


    output = conv1(features)
    output=pool1(output)
    output = conv2(output)
    output=pool2(output)
    linear_in_features=output.shape[1]*output.shape[2]
    print(linear_in_features)

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv1d(in_channels=3, out_channels=num_filters1, kernel_size=2, stride=1)
            self.pool = nn.MaxPool1d(kernel_size=2)
            self.bn1 = nn.BatchNorm1d(100)
            self.conv2 = nn.Conv1d(in_channels=num_filters1, out_channels=num_filters2, kernel_size=2, stride=1)
            self.bn2 = nn.BatchNorm1d(48)
            self.fc1 = nn.Linear(in_features=linear_in_features,out_features=20)
            self.softmax = nn.Softmax(dim=1)
            self.relu = nn.ReLU()
            self.drop = nn.Dropout(0.5)

        def forward(self, x):
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.bn1(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.bn2(x)
            x = self.drop(x)
            x = x.view(-1,linear_in_features)
            x = self.fc1(x)
            #x = self.softmax(x)
            return x
    model=Net()
    model.to(device)
    return model
   

In [132]:
model=get_model(num_filters1=100,num_filters2=48)
model.load_state_dict(torch.load('/scratch/lnw8px/GAN/models/CNN_noGAN.pt'))

288


<All keys matched successfully>

In [117]:
#model.eval()
'''
for CNN_noGAN this gave 0.59515 and std of 0.024
for CNN this gave 0.63175 and std of 0.024
'''
acc_list=[]
for i in range(0,100):
    acc=get_test_acc_loss()
    acc_list.append(acc)
np.mean(np.array(acc_list)),np.std(np.array(acc_list))

(0.63175, 0.024076700355322802)

In [88]:
def get_test_acc_loss():
    features,labels=get_batch(data,batch_size=len(data))
    pred=model(features)
    predmax=torch.max(pred,1)[1]
    iscorrect=(predmax==labels)
    num_correct=torch.sum(iscorrect).item()
    num_total=iscorrect.shape[0]
    acc=num_correct/num_total
    return acc

In [106]:
get_test_acc_loss()

0.58

In [118]:
def get_per_class_acc(test_data):
    features,labels=get_batch(test_data,batch_size=len(test_data))
    pred=model(features)
    predmax=torch.max(pred,1)[1]
    iscorrect=(predmax==labels)
    iscorrect=iscorrect.cpu().numpy()
    predmax=predmax.cpu().numpy()
    ones=np.ones(len(iscorrect))
    res=np.bincount(predmax,iscorrect)
    counts=np.bincount(predmax,ones)
    perclass_acc=res/counts
    return perclass_acc

In [136]:
'''
for CNN model :
array([0.86738095, 0.61834776, 0.58060924, 0.87063492, 0.56460905,
       0.65929293, 0.54892143, 0.45047425, 0.86713953, 0.6697619 ,
       0.84944444, 0.39105339, 0.89689394, 0.53751885, 0.295     ,
       0.0425    , 0.75385448, 0.6522409 , 0.69869048, 0.7228022 ])

for CNN_noGAN :
array([0.84976107, 0.79464646, 0.56018093, 0.55      , 0.69961566,
       0.49438312, 0.32331811, 0.5128268 , 0.68390776, 0.83876263,
       0.7940873 , 0.22178571, 0.95277778, 0.77925685, 0.        ,
       0.06100289, 0.59376751, 0.55987374, 0.70896825, 0.89222222])
'''

acc=get_per_class_acc(data)
for i in range(9):
    acc+=get_per_class_acc(data)
acc=acc/10
acc

array([0.84976107, 0.79464646, 0.56018093, 0.55      , 0.69961566,
       0.49438312, 0.32331811, 0.5128268 , 0.68390776, 0.83876263,
       0.7940873 , 0.22178571, 0.95277778, 0.77925685, 0.        ,
       0.06100289, 0.59376751, 0.55987374, 0.70896825, 0.89222222])